# Multi-Agent Systems
In a multi-agent system individual agents 
 - with different personas, 
 - tool sets can 
 - communicate, 
 - collaborate, 
 - add additional context, 
 - and do work in parallel. 

They often have special rules for how to work as a team, decide things together, and handle disagreements.

# Install Packages

In [ ]:
#Install packages
%pip install pyautogen
%pip install openai
%pip install chroma
%pip install python-dotenv
%pip install markdownify
%pip install apify_client

# Initialization 

In [3]:
import os
import autogen
from dotenv import load_dotenv

load_dotenv()
# Save your OPENAI_API_KEY in a .env file in the same directory as this script
llm_config_dict = {"config_list": [{"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}]}

# Two-agent chat: 
 - The simplest form of conversation pattern where two agents chat with each other using Autogen.

<img src="./images/autogen_image1.png" alt="GroupChat Flow" width="500" style="display: block; margin-left: auto; margin-right: auto;">


In [4]:
import os

from autogen import ConversableAgent

student_agent = ConversableAgent(
    name="Student_Agent",
    system_message="You are a student willing to learn. You ask meaningful and precise follow up questions and are eager to learn more.",
    llm_config=llm_config_dict,
)
teacher_agent = ConversableAgent(
    name="Teacher_Agent",
    system_message="You are an expert at Large Language model research and you teach cutting edge AI development technologies.",
    llm_config=llm_config_dict,
)

chat_result = student_agent.initiate_chat(
    teacher_agent,
    message="How to prove or test that Artificial Intelligence is actually intelligent?",
    summary_method="reflection_with_llm",
    max_turns=2,
)

[autogen.oai.client: 09-30 12:28:43] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-30 12:28:43] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
Student_Agent (to Teacher_Agent):

How to prove or test that Artificial Intelligence is actually intelligent?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Teacher_Agent (to Student_Agent):

Proving or testing that an artificial intelligence (AI) system is genuinely "intelligent" is a multifaceted challenge that can vary depending on the definition of intelligence you're using. Here are several approaches and methodologies commonly used to test and evaluate AI systems:

### 1. **Turing Test**
- **Description**: Proposed by Alan Turing in 1950, the Turing Test evaluates a machine's ability to exhibit intelligent behavior equivale

Let's look at the summary. It is contained within the chat_result object of type ChatResult, which was returned by the initiate_chat method.

In [33]:
print(chat_result.summary)

The conversation outlined various methods for testing AI intelligence, including the Turing Test, benchmark tests, CAPTCHA, ethics and emotional intelligence tests, cognitive architecture evaluations, adversarial testing, real-world environment testing, human-level performance benchmarks, explainability assessments, and adaptability tests. Challenges in keeping these tests relevant include rapid AI advancement, changing definitions of intelligence, diverse applications, and biases. Strategies to address these challenges involve continuous updates to standards, dynamic benchmarks, cross-domain evaluations, bias audits, inclusive datasets, user-centric evaluations, lifelong learning tests, transfer learning evaluations, and open-source platforms. Future directions suggest integrating insights from neuroscience and cognitive science, evaluating real-world impacts, and combining human and automated evaluations.


In the above example, the summary method is configured to `reflection_with_llm`, which processes a list of conversation messages and summarizes them by invoking an LLM.

Initially, the summary method attempts to utilize the recipient’s LLM; if it is unavailable, it defaults to the sender’s LLM.

Here, the recipient is “Teacher_Agent” and the sender is “Student_Agent”.

The input prompt for the LLM is the following default prompt:

In [34]:
print(ConversableAgent.DEFAULT_SUMMARY_PROMPT)

Summarize the takeaway from the conversation. Do not add any introductory phrases.


# Sequential chat: 
 - a series of conversations between two agents, linked by a carryover mechanism that transfers the summary of the prior chat to the context of the subsequent chat.

<img src="./images/autogen_image2.png" alt="GroupChat Flow" width="500" style="display: block; margin-left: auto; margin-right: auto;">


In [35]:
from autogen import ConversableAgent

report_agent = ConversableAgent(
    name="Report_Agent",
    system_message="You are responsible for creating a report by extracting insights from the chat history.",
    llm_config=llm_config_dict,
    human_input_mode="NEVER",
)

# The Researcher Agent discovers the potential of AGI.
researcher_agent = ConversableAgent(
    name="Researcher_Agent",
    system_message="You explore and describe the potential capabilities and advancements of AGI.",
    llm_config=llm_config_dict,
    human_input_mode="NEVER",
)

# The Ethicist Agent evaluates the ethical implications of AGI.
ethicist_agent = ConversableAgent(
    name="Ethicist_Agent",
    system_message="You evaluate the ethical implications of AGI based on the research findings.",
    llm_config=llm_config_dict,
    human_input_mode="NEVER",
)

# The Economist Agent analyzes the economic impact of AGI.
economist_agent = ConversableAgent(
    name="Economist_Agent",
    system_message="You analyze the economic impact of AGI based on the ethical evaluations.",
    llm_config=llm_config_dict,
    human_input_mode="NEVER",
)

# The Policy Maker Agent develops policies based on the findings of the previous agents.
policy_maker_agent = ConversableAgent(
    name="Policy_Maker_Agent",
    system_message="You develop policies to manage AGI based on the economic analysis.",
    llm_config=llm_config_dict,
    human_input_mode="NEVER",
)





[autogen.oai.client: 09-30 09:51:06] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-30 09:51:07] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-30 09:51:07] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-30 09:51:08] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-30 09:51:08] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [36]:
# Start a sequence of two-agent chats.
# Each element in the list is a dictionary that specifies the arguments
# for the initiate_chat method.

chat_results = report_agent.initiate_chats(
    [
        {
            "recipient": researcher_agent,
            "message": "Discover the potential of AGI",
            "max_turns": 1,
            "summary_method": "last_msg",
        },
        {
            "recipient": ethicist_agent,
            "message": "Evaluate the ethical implications of AGI",
            "max_turns": 1,
            "summary_method": "last_msg",
        },
        {
            "recipient": economist_agent,
            "message": "Analyze the economic impact of AGI",
            "max_turns": 1,
            "summary_method": "last_msg",
        },
        {
            "recipient": policy_maker_agent,
            "message": "Develop policies to manage AGI",
            "max_turns": 1,
            "summary_method": "last_msg",
        }
    ]
)



********************************************************************************
Starting a new chat....

********************************************************************************
Report_Agent (to Researcher_Agent):

Discover the potential of AGI

--------------------------------------------------------------------------------
Researcher_Agent (to Report_Agent):

## Potential Capabilities and Advancements of Artificial General Intelligence (AGI)

### Overview
Artificial General Intelligence (AGI) represents a significant leap from current AI, known as Narrow AI, which is designed to perform specific tasks. AGI, also known as Strong AI, aims to replicate the breadth and depth of human cognitive abilities, allowing it to understand, learn, and apply knowledge across a wide range of domains without human intervention. The realization of AGI could fundamentally transform various sectors and aspects of human life.

### Potential Capabilities

#### 1. **Autonomous Learning**
AGI syst

In [37]:
print("First Chat Summary: ", chat_results[0].summary)
print("Second Chat Summary: ", chat_results[1].summary)
print("Third Chat Summary: ", chat_results[2].summary)
print("Fourth Chat Summary: ", chat_results[3].summary)

First Chat Summary:  ## Potential Capabilities and Advancements of Artificial General Intelligence (AGI)

### Overview
Artificial General Intelligence (AGI) represents a significant leap from current AI, known as Narrow AI, which is designed to perform specific tasks. AGI, also known as Strong AI, aims to replicate the breadth and depth of human cognitive abilities, allowing it to understand, learn, and apply knowledge across a wide range of domains without human intervention. The realization of AGI could fundamentally transform various sectors and aspects of human life.

### Potential Capabilities

#### 1. **Autonomous Learning**
AGI systems would be capable of learning autonomously without the need for constant supervision or large, labeled datasets. By observing environments, experiences, and interactions, AGI could identify patterns and acquire skills similarly to how humans learn.

#### 2. **General Problem Solving**
Unlike Narrow AI, which is typically limited to specific tasks, 

# Tool Use
In this section, we'll demonstrate tool-use capability of the agentic system. Web Search and RAG are popular capabilities used by people every day, we show how to use AutoGen to do:

* Web search using Bing API. 
* Web scraping using Apify API
* Question and answering with RAG.

More info can be found here : https://microsoft.github.io/autogen/docs/tutorial/tool-use 

## Web Search using Bing API

In [5]:
# Put your Bing API key in the .env file in the same directory as this script
BING_API_KEY = os.environ["BING_API_KEY"]

llm_config = {
    "timeout": 600,
    "cache_seed": 44,  # change the seed for different trials
    "config_list": llm_config_dict['config_list'],
    "temperature": 0,
}

summarizer_llm_config = {
    "timeout": 600,
    "cache_seed": 44,  # change the seed for different trials
    "config_list": llm_config_dict['config_list'],
    "temperature": 0,
}

In [6]:
from autogen.agentchat.contrib.web_surfer import WebSurferAgent

web_surfer = WebSurferAgent(
    "web_surfer",
    llm_config=llm_config,
    summarizer_llm_config=summarizer_llm_config,
    browser_config={"viewport_size": 4096, "bing_api_key": BING_API_KEY},
)

user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    code_execution_config=False,
    default_auto_reply="",
    is_termination_msg=lambda x: True,
)

[autogen.oai.client: 09-30 12:38:55] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-30 12:38:56] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-30 12:38:56] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-30 12:38:57] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-30 12:38:57] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-30 12:38:58] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-30 12:38:59] {184} WARNING - The API key specified is not a valid OpenAI format; i

In [1]:
task1 = """
What is Artificial General Intelligence ?
"""

user_proxy.initiate_chat(web_surfer, message=task1, max_turns=1)

user_proxy (to web_surfer):


What is Artificial General Intelligence ?


--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION informational_web_search...
web_surfer (to user_proxy):

Address: bing: What is Artificial General Intelligence
Title: What is Artificial General Intelligence - Search
Viewport position: Showing page 1 of 1.
A Bing search for 'What is Artificial General Intelligence' found 15 results:

## Web Results
1. [Artificial general intelligence - Wikipedia](https://en.wikipedia.org/wiki/Artificial_general_intelligence)
AGI is a theoretical type of AI that matches or surpasses human capabilities across a wide range of cognitive tasks. Learn about its definition, characteristics, tests, history, and challenges from this comprehensive article.

2. [A.I. Rising](https://en.wikipedia.org/wiki/A.I._Rising)
A.I. Rising (also known as Ederlezi [a] Rising and Ederlezi ébredése) is a 2018 Engli

ChatResult(chat_id=None, chat_history=[{'content': '\nWhat is Artificial General Intelligence ?\n', 'role': 'assistant', 'name': 'user_proxy'}, {'content': "Address: bing: What is Artificial General Intelligence\nTitle: What is Artificial General Intelligence - Search\nViewport position: Showing page 1 of 1.\n=======================\nA Bing search for 'What is Artificial General Intelligence' found 15 results:\n\n## Web Results\n1. [Artificial general intelligence - Wikipedia](https://en.wikipedia.org/wiki/Artificial_general_intelligence)\nAGI is a theoretical type of AI that matches or surpasses human capabilities across a wide range of cognitive tasks. Learn about its definition, characteristics, tests, history, and challenges from this comprehensive article.\n\n2. [A.I. Rising](https://en.wikipedia.org/wiki/A.I._Rising)\nA.I. Rising (also known as Ederlezi [a] Rising and Ederlezi ébredése) is a 2018 English-language Serbian science fiction film directed by Lazar Bodroža and based on

In [57]:
task2 = "Click on the first link."
user_proxy.initiate_chat(web_surfer, message=task2)

user_proxy (to web_surfer):

Click on the first link.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION page_down...
web_surfer (to user_proxy):

Address: bing: What is Artificial General Intelligence
Title: What is Artificial General Intelligence - Search
Viewport position: Showing page 1 of 1.
A Bing search for 'What is Artificial General Intelligence' found 15 results:

## Web Results
1. [Artificial general intelligence - Wikipedia](https://en.wikipedia.org/wiki/Artificial_general_intelligence)
AGI is a theoretical type of AI that matches or surpasses human capabilities across a wide range of cognitive tasks. Learn about its definition, characteristics, tests, history, and challenges from this comprehensive article.

2. [A.I. Rising](https://en.wikipedia.org/wiki/A.I._Rising)
A.I. Rising (also known as Ederlezi [a] Rising and Ederlezi ébredése) is a 2018 English-language Serbian science fictio

ChatResult(chat_id=None, chat_history=[{'content': 'Click on the first link.', 'role': 'assistant', 'name': 'user_proxy'}, {'content': "Address: bing: What is Artificial General Intelligence\nTitle: What is Artificial General Intelligence - Search\nViewport position: Showing page 1 of 1.\n=======================\nA Bing search for 'What is Artificial General Intelligence' found 15 results:\n\n## Web Results\n1. [Artificial general intelligence - Wikipedia](https://en.wikipedia.org/wiki/Artificial_general_intelligence)\nAGI is a theoretical type of AI that matches or surpasses human capabilities across a wide range of cognitive tasks. Learn about its definition, characteristics, tests, history, and challenges from this comprehensive article.\n\n2. [A.I. Rising](https://en.wikipedia.org/wiki/A.I._Rising)\nA.I. Rising (also known as Ederlezi [a] Rising and Ederlezi ébredése) is a 2018 English-language Serbian science fiction film directed by Lazar Bodroža and based on a 1980s short story 

In [10]:
task3 = "What is the difference beween AGI and classic AI?"
user_proxy.initiate_chat(web_surfer, message=task3)

user_proxy (to web_surfer):

What is the difference beween AGI and classic AI

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION informational_web_search...
web_surfer (to user_proxy):

Address: bing: difference between AGI and classic AI
Title: difference between AGI and classic AI - Search
Viewport position: Showing page 1 of 1.
A Bing search for 'difference between AGI and classic AI' found 12 results:

## Web Results
1. [AI Vs AGI: What's The Difference? - Forbes](https://www.forbes.com/councils/forbestechcouncil/2018/09/17/ai-vs-agi-whats-the-difference/)
Others, though, would claim there is a distinct difference. It might be easy to think about AI as a broad field, while AGI is a more specific focus within it. General AI applies some of the same ...

2. [The Important Difference Between Generative AI And AGI - Forbes](https://www.forbes.com/sites/bernardmarr/2024/05/08/the-important-differen

ChatResult(chat_id=None, chat_history=[{'content': 'What is the difference beween AGI and classic AI', 'role': 'assistant', 'name': 'user_proxy'}, {'content': "Address: bing: difference between AGI and classic AI\nTitle: difference between AGI and classic AI - Search\nViewport position: Showing page 1 of 1.\n=======================\nA Bing search for 'difference between AGI and classic AI' found 12 results:\n\n## Web Results\n1. [AI Vs AGI: What's The Difference? - Forbes](https://www.forbes.com/councils/forbestechcouncil/2018/09/17/ai-vs-agi-whats-the-difference/)\nOthers, though, would claim there is a distinct difference. It might be easy to think about AI as a broad field, while AGI is a more specific focus within it. General AI applies some of the same ...\n\n2. [The Important Difference Between Generative AI And AGI - Forbes](https://www.forbes.com/sites/bernardmarr/2024/05/08/the-important-difference-between-generative-ai-and-agi/)\nHere are the key differences: Capability: Gene

## Web Scraping Using Apify Tool

In [11]:
# Put your Apify API key in the .env file in the same directory as this script
APIFY_API_KEY = os.environ["APIFY_API_KEY"]

In [12]:
from apify_client import ApifyClient
from typing_extensions import Annotated


def scrape_page(url: Annotated[str, "The URL of the web page to scrape"]) -> Annotated[str, "Scraped content"]:
    # Initialize the ApifyClient with your API token
    client = ApifyClient(token=APIFY_API_KEY)

    # Prepare the Actor input
    run_input = {
        "startUrls": [{"url": url}],
        "useSitemaps": False,
        "crawlerType": "playwright:firefox",
        "includeUrlGlobs": [],
        "excludeUrlGlobs": [],
        "ignoreCanonicalUrl": False,
        "maxCrawlDepth": 0,
        "maxCrawlPages": 1,
        "initialConcurrency": 0,
        "maxConcurrency": 200,
        "initialCookies": [],
        "proxyConfiguration": {"useApifyProxy": True},
        "maxSessionRotations": 10,
        "maxRequestRetries": 5,
        "requestTimeoutSecs": 60,
        "dynamicContentWaitSecs": 10,
        "maxScrollHeightPixels": 5000,
        "removeElementsCssSelector": """nav, footer, script, style, noscript, svg,
    [role=\"alert\"],
    [role=\"banner\"],
    [role=\"dialog\"],
    [role=\"alertdialog\"],
    [role=\"region\"][aria-label*=\"skip\" i],
    [aria-modal=\"true\"]""",
        "removeCookieWarnings": True,
        "clickElementsCssSelector": '[aria-expanded="false"]',
        "htmlTransformer": "readableText",
        "readableTextCharThreshold": 100,
        "aggressivePrune": False,
        "debugMode": True,
        "debugLog": True,
        "saveHtml": True,
        "saveMarkdown": True,
        "saveFiles": False,
        "saveScreenshots": False,
        "maxResults": 9999999,
        "clientSideMinChangePercentage": 15,
        "renderingTypeDetectionPercentage": 10,
    }

    # Run the Actor and wait for it to finish
    run = client.actor("aYG0l9s7dbB7j3gbS").call(run_input=run_input)

    # Fetch and print Actor results from the run's dataset (if there are any)
    text_data = ""
    for item in client.dataset(run["defaultDatasetId"]).iterate_items():
        text_data += item.get("text", "") + "\n"

    average_token = 0.75
    max_tokens = 20000  # slightly less than max to be safe 32k
    text_data = text_data[: int(average_token * max_tokens)]
    return text_data

In [13]:
from autogen import ConversableAgent, register_function

# Create web scrapper agent.
scraper_agent = ConversableAgent(
    "WebScraper",
    llm_config=llm_config_dict,
    system_message="You are a web scrapper and you can scrape any web page using the tools provided. "
    "Returns 'TERMINATE' when the scraping is done.",
)

# Create user proxy agent.
user_proxy_agent = ConversableAgent(
    "UserProxy",
    llm_config=False,  # No LLM for this agent.
    human_input_mode="NEVER",
    code_execution_config=False,  # No code execution for this agent.
    is_termination_msg=lambda x: x.get("content", "") is not None and "terminate" in x["content"].lower(),
    default_auto_reply="Please continue if not finished, otherwise return 'TERMINATE'.",
)

# Register the function with the agents.
register_function(
    scrape_page,
    caller=scraper_agent,
    executor=user_proxy_agent,
    name="scrape_page",
    description="Scrape a web page and return the content.",
)

[autogen.oai.client: 09-30 12:57:38] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-30 12:57:38] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [28]:
url = 'https://en.wikipedia.org/wiki/Artificial_general_intelligence'
chat_result = user_proxy_agent.initiate_chat(
    scraper_agent,
    message=f"Can you scrape {url} for me?")
    #summary_method="reflection_with_llm")

UserProxy (to WebScraper):

Can you scrape https://en.wikipedia.org/wiki/Artificial_general_intelligence for me?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
WebScraper (to UserProxy):

***** Suggested tool call (call_0nIPIvLcJF7Wk1hVmXrN5X5M): scrape_page *****
Arguments: 
{"url":"https://en.wikipedia.org/wiki/Artificial_general_intelligence"}
****************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION scrape_page...
UserProxy (to WebScraper):

UserProxy (to WebScraper):

***** Response from calling tool (call_0nIPIvLcJF7Wk1hVmXrN5X5M) *****
Artificial general intelligence - Wikipedia
Artificial general intelligence (AGI) is a type of artificial intelligence (AI) that falls within the lower and upper limits of human cognitive capabilities across a wide range of cognitive tasks.[1][2][3][4][5][

In [29]:
agi_wiki = chat_result.chat_history[2]['content']

In [30]:
agi_wiki

'Artificial general intelligence - Wikipedia\nArtificial general intelligence (AGI) is a type of artificial intelligence (AI) that falls within the lower and upper limits of human cognitive capabilities across a wide range of cognitive tasks.[1][2][3][4][5][6][7] \nThis contrasts with narrow AI, which is limited to specific tasks.[8][9] Artificial superintelligence (ASI) refers to types of intelligence that range from being only marginally smarter than the upper limits of human intelligence to greatly exceeding human cognitive capabilities by orders of magnitude.[10] AGI is considered one of the definitions of strong AI.[11] \nAGI may be comparable to, match, differ from, or even appear alien-like relative to human intelligence, encompassing a spectrum of possible cognitive architectures and capabilities that includes the spectrum of human-level intelligence.[12][13][14] \nCreating AGI is a primary goal of AI research and of companies such as OpenAI[15] and Meta.[16] A 2020 survey iden

In [31]:
with open("agi_wiki_content.txt", "w", encoding="utf-8") as file:
    file.write(agi_wiki)

## Perform RAG on the Scraped Wiki File

In [32]:
# from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
import chromadb

DOC_PATH = ["./agi_wiki_content.txt"]

In [34]:
llm_config = {'config_list': llm_config_dict['config_list'], 'timeout': 60, 'cache_seed': 88, 'temperature': 0}

In [35]:
assistant = AssistantAgent(
    name="assistant",
    human_input_mode="NEVER",
    system_message="You are a helpful assistant.",
    llm_config=llm_config
)
        
ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    retrieve_config={
        "task": "qa",
        "docs_path": DOC_PATH,
        "chunk_token_size": 1000,
        "model": llm_config_dict["config_list"][0]["model"],
        "client": chromadb.PersistentClient(path="/tmp/chromadb"),
        "collection_name": "boise_idaho",
        "get_or_create": True, # if True, will create/return a collection for the retrieve chat.
    },
    code_execution_config={"use_docker": False}
)

[autogen.oai.client: 09-30 13:34:53] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


c:\Users\jiech\AppData\Local\miniconda3\envs\cikm\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\jiech\AppData\Local\miniconda3\envs\cikm\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [53]:
prompt = "What is the population of Boise, Idaho?"
res = ragproxyagent.initiate_chat(assistant, message=ragproxyagent.message_generator, problem=prompt, n_results=1, silent=True)

2024-09-30 09:53:07,467 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Use the existing collection `boise_idaho`.
max_tokens is too small to fit a single line of text. Breaking this line:
		 ...
Failed to split docs with must_break_at_empty_line being True, set to False.
2024-09-30 09:53:07,484 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Found 7 chunks.
2024-09-30 09:53:07,487 - autogen.agentchat.contrib.vectordb.chromadb - INFO - No content embedding is provided. Will use the VectorDB's embedding function to generate the content embedding.


Trying to create collection.
VectorDB returns doc_ids:  [['e495c059']]
Adding content of doc e495c059 to context.


In [54]:
print(f"Question:{prompt}")
print(f"Answer from RAG: {res.chat_history[-1]['content']}")

Question:What is the population of Boise, Idaho?
Answer from RAG: The population of Boise, Idaho is 235,684 as of the 2020 census.


# Group Chat 

Up to this point, we have encountered conversation patterns that include either two agents or a series of two-agent interactions. 

AutoGen introduces a broader conversation model known as **group chat**, which includes more than two agents. 

The fundamental concept of group chat is that all agents participate in a unified conversation thread and share the same context. 

This approach is beneficial for tasks that necessitate collaboration among several agents.


<img src="./images/autogen_image3.png" alt="GroupChat Flow" width="500" style="display: block; margin-left: auto; margin-right: auto;">



The Group Chat Manager orchestrates a group chat with the following steps:

1. The Group Chat Manager selects an agent to speak.
2. The selected agent speaks and the message is sent back to the Group Chat Manager.
3. The Group Chat Manager broadcasts the message to all other agents in the group.
4. This process repeats until the conversation stops.

Group Chat Manager can use several strategies to select the next agent :

- **round_robin**: The Group Chat Manager selects agents in a round-robin fashion based on the order of the agents provided.
- **random**: The Group Chat Manager selects agents randomly.
- **manual**: The Group Chat Manager selects agents by asking for human input.
- **auto**: The default strategy, which selects agents using the Group Chat Manager’s LLM.

In [55]:
import os
from autogen import ConversableAgent, GroupChat, GroupChatManager, UserProxyAgent
from dotenv import load_dotenv

load_dotenv()
user_proxy = UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={
        "last_n_messages": 2,
        "work_dir": "groupchat",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    human_input_mode="TERMINATE",
)

economist = ConversableAgent(
    name="Economist",
    system_message="You are a highly experienced economist specializing in global economic trends and geopolitical risks.",
    llm_config=llm_config_dict,
    description="Expert in global economics and the impact of conflicts."
)

capitalist = ConversableAgent(
    name="Capitalist",
    system_message="You are a pragmatic capitalist focused on market opportunities and risks, even in extreme circumstances.",
    llm_config=llm_config_dict,
    description="Astute observer of market dynamics amidst global crises."
)

environmentalist = ConversableAgent(
    name="Environmentalist",
    system_message="You are an environmentalist advocating for sustainable practices and conservation of natural resources.",
    llm_config=llm_config_dict,
    description="Promotes environmental awareness and sustainable solutions."
)

# Group Chat Setup
group_chat = GroupChat(
    agents=[user_proxy, economist, capitalist, environmentalist],
    messages=[],
    max_round=5,
    send_introductions=True,
)

# Group Chat Manager
group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config_dict
)

# Initiate the Conversation
chat_result = user_proxy.initiate_chat(
    group_chat_manager,
    message="Analyze the potential global ramifications of onset of Artificial general intelligence. Summerise your insights.",
    # summary_method="reflection_with_llm"
)

# Print the Discussion Summary Report
print(chat_result.summary)

[autogen.oai.client: 09-30 09:53:09] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-30 09:53:09] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-30 09:53:09] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-30 09:53:10] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
User_proxy (to chat_manager):

Analyze the potential global ramifications of onset of Artificial general intelligence. Summerise your insights.

--------------------------------------------------------------------------------
[autogen.oai.client: 09-30 09:53:10] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.

Next speaker: Economist


>>

Please refer to https://microsoft.github.io/autogen/docs/ for information.